### 1.1 Data Preprocessing (Families)

In [4]:
import pandas as pd
import numpy as np
import torch
import os
import sys
import configparser
import json

In [5]:
config_path = "../../config/main.conf"
conf = configparser.ConfigParser()
conf.read(config_path)

model_conf = configparser.ConfigParser()
model_conf.read(conf['path']['model'])

['../../config/model.conf']

In [7]:
data_partitions_dirpath = conf['path']['data_part']
print('Available dataset partitions: ', os.listdir(data_partitions_dirpath))

Available dataset partitions:  ['train', 'dev', 'contact_maps', 'GO', '.ipynb_checkpoints', 'test', 'download.sh']


In [8]:
%%time
def read_all_shards(partition='dev', data_dir=data_partitions_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    return pd.concat(shards)

test = read_all_shards('test')
dev = read_all_shards('dev')
train = read_all_shards('train')

partitions = {'test': test, 'dev': dev, 'train': train}
for name, df in partitions.items():
    print('Dataset partition "%s" has %d sequences' % (name, len(df)))

Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences
CPU times: user 6.38 s, sys: 1.41 s, total: 7.8 s
Wall time: 16.4 s


In [13]:
SAMPLE_RATE = 10

fams = np.array(train["family_accession"].value_counts().index)[::SAMPLE_RATE]

In [14]:
fam_list = ','.join(fams)
with open('../../data/contact_maps/pfam_list.txt', 'w') as file:
    file.write(fam_list)

### 1.2 Contact Maps

In [6]:
import numpy as np
import Bio.PDB
import os 
import scipy.sparse

In [7]:
config_path = "../../config/contact_maps.conf"
conf = configparser.ConfigParser()
conf.read(config_path)

filename = conf['path']['pdb_codes']
pdb_dir = conf['path']['pdb_dir']
contact_map_dir = conf['path']['contact_maps_dir']
sequence_dir = conf['path']['sequence_dir']

In [8]:
def calc_residue_dist(residue_one, residue_two, aa_cache) :
    """Returns the C-alpha distance between two residues"""       
    if aa_cache[residue_one.resname] and aa_cache[residue_two.resname]:
        diff_vector  = residue_one["CA"].coord - residue_two["CA"].coord
        return np.sqrt(np.sum(diff_vector * diff_vector))
    else:
        return 0
        
def calc_dist_matrix(chain, aa_cache) :
    """Returns a matrix of C-alpha distances between two chains"""
    answer = np.zeros((len(chain), len(chain)), np.float)
    for row, residue_one in enumerate(chain) :
        for col, residue_two in enumerate(chain) :
            answer[row, col] = calc_residue_dist(residue_one, residue_two, aa_cache)
    return answer


In [9]:
def calc_dist_matrix_new(sequence):
    X = []
    for res in sequence:
        X.append(res['CA'].coord)
    X = np.array(X)
    e = np.ones(X.shape[0]).reshape(-1, 1)
    s = np.sum(np.square(X), axis = 1).reshape(-1, 1)
    dist = np.sqrt((s @ e.T) + (e @ s.T) - (2 * X @ X.T))
    return dist

In [10]:
%%time
sequence_dict = {}
pdb_file = open(filename, 'r')
pdb_list = pdb_file.read()
num_successful_maps = 0
num_failed_maps = 0
for pdb_code in pdb_list.split(', ')[0:1]: #change to all codes when deployed
    pdb_code = '101M'
    pdbl = Bio.PDB.PDBList()
    pdb_path = pdbl.retrieve_pdb_file(pdb_code, pdir = pdb_dir, file_format = 'pdb', overwrite = True)
    structure = Bio.PDB.PDBParser(QUIET = True).get_structure(pdb_code, pdb_path)
    model = structure[0]
    sequence = Bio.PDB.Selection.unfold_entities(model, 'R')
    if len(sequence) < 3000:
        try:
            ppb = Bio.PDB.CaPPBuilder()
            for pp in ppb.build_peptides(structure):
                sequence_dict[pdb_code] = str(pp.get_sequence())
            dist_matrix = calc_dist_matrix_new(pp)
            contact_map = np.array((dist_matrix < 8.0) & (dist_matrix > 0.01))*1
            sparse_contact_map = scipy.sparse.coo_matrix(contact_map)
            print(sparse_contact_map.shape)
            scipy.sparse.save_npz(contact_map_dir + '/' + pdb_code + '.npz', sparse_contact_map)
            num_successful_maps += 1
        except:
            e = sys.exc_info()[0]
            print('Error for PDB code {}'.format(pdb_code), e)
            num_failed_maps += 1
    else:
        print('Skipping PDB code {} because it is too long'.format(pdb_code))
        num_failed_maps += 1
    
    os.remove(pdb_path)
    
json_data = json.dumps(sequence_dict)
f = open(sequence_dir,"w")
f.write(json_data)
f.close()
        
print('Number of maps generated: {}'.format(num_successful_maps))
print('Number of pdb codes skipped: {}'.format(num_failed_maps))

(154, 154)
Number of maps generated: 1
Number of pdb codes skipped: 0
CPU times: user 59.3 ms, sys: 13 ms, total: 72.3 ms
Wall time: 267 ms


/global/homes/n/nswenson/.conda/envs/biopy/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  
/global/homes/n/nswenson/.conda/envs/biopy/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less
/global/homes/n/nswenson/.conda/envs/biopy/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater


In [54]:
import json

json_data = json.dumps(sequence_dict)
f = open("dict.json","w")
f.write(json_data)
f.close()

In [56]:
with open("dict.json","r") as f:
    sequence_dict2 = json.load(f)
    f.close()
sequence_dict2

{'101M': 'MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRVKHLKTEAEMKASEDLKKHGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG'}

In [46]:
try:
    os.mkdir('../data/contact_maps/Contact Maps')
except:
    pass

In [40]:
os.rmdir('../data/contact_maps/CM')

In [83]:
len(pdb_list)

18682

In [17]:
%%time
structure = Bio.PDB.PDBParser(QUIET = True).get_structure(pdb_code, pdb_path)

CPU times: user 340 ms, sys: 0 ns, total: 340 ms
Wall time: 379 ms


In [8]:
%%time
import datetime
print(datetime.datetime.now())
model = structure[0]
print(datetime.datetime.now())
sequence = Bio.PDB.Selection.unfold_entities(model, 'R')
print(datetime.datetime.now())
dist_matrix = calc_dist_matrix(sequence, aa_cache)
print(datetime.datetime.now())
contact_map = np.array((dist_matrix < 8.0) & (dist_matrix > 0.01))*1
print(datetime.datetime.now())
len(sequence)

2020-02-06 16:50:16.027645
2020-02-06 16:50:16.027811
2020-02-06 16:50:16.028224
2020-02-06 16:50:35.347644
2020-02-06 16:50:35.365070
CPU times: user 19.3 s, sys: 27.9 ms, total: 19.3 s
Wall time: 19.3 s


1630

In [104]:
%%time
aa_cache = {}
for res in sequence:
    if res.resname not in aa_cache:
        aa_cache[res.resname] = Bio.PDB.is_aa(res)

CPU times: user 783 µs, sys: 14 µs, total: 797 µs
Wall time: 806 µs


In [20]:
%%time
diff_vector  = sequence[0]["CA"].coord - sequence[1]["CA"].coord

CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 29.8 µs


In [9]:
%%time
#sequence = Bio.PDB.Selection.unfold_entities(model, 'R')
dist_matrix = calc_dist_matrix(sequence, aa_cache)
#contact_map = np.array((dist_matrix < 8.0) & (dist_matrix > 0.01))*1
#sparse_contact_map = scipy.sparse.coo_matrix(contact_map)
#scipy.sparse.save_npz(contact_map_dir + '/' + pdb_code + '.npz', sparse_contact_map)

CPU times: user 19.5 s, sys: 16 ms, total: 19.5 s
Wall time: 19.5 s


In [31]:
%%time
diff_vector = sequence[0]['CA'].coord -  sequence[1]['CA'].coord

CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37.4 µs


In [32]:
%%time
aa_cache[sequence[0].resname]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.91 µs


True

In [33]:
%%time
np.sqrt(np.sum(diff_vector * diff_vector))

CPU times: user 79 µs, sys: 0 ns, total: 79 µs
Wall time: 83 µs


3.805978

In [35]:
%%time
np.sqrt(np.sum(np.square(diff_vector)))

CPU times: user 158 µs, sys: 0 ns, total: 158 µs
Wall time: 163 µs


3.805978

In [38]:
%%time
np.linalg.norm(diff_vector)

CPU times: user 125 µs, sys: 0 ns, total: 125 µs
Wall time: 87.7 µs


3.805978

In [39]:
%%time
np.linalg.norm(sequence[0]['CA'].coord -  sequence[1]['CA'].coord)

CPU times: user 150 µs, sys: 0 ns, total: 150 µs
Wall time: 111 µs


3.805978

In [41]:
diff_vector

array([ 0.32000065, -1.3210001 , -3.5550003 ], dtype=float32)

In [46]:
%%time
diff_vector_int = diff_vector.astype(int)
diff_vector_int

CPU times: user 46 µs, sys: 1e+03 ns, total: 47 µs
Wall time: 49.1 µs


array([ 0, -1, -3])

In [45]:
%%time
np.sqrt(np.sum(diff_vector_int * diff_vector_int))

CPU times: user 185 µs, sys: 2 µs, total: 187 µs
Wall time: 193 µs


3.1622776601683795

In [5]:
pdb_code = '1A28'
pdbl = Bio.PDB.PDBList()
pdb_path = pdbl.retrieve_pdb_file(pdb_code, pdir = pdb_dir, file_format = 'pdb', overwrite = True)

In [207]:
%%time
old_matrix = calc_dist_matrix(sequence, aa_cache)
contact_map_old = np.array((old_matrix < 8.0) & (old_matrix > 0.1))*1

CPU times: user 2.26 s, sys: 7.13 ms, total: 2.26 s
Wall time: 2.26 s


In [208]:
%%time
new_matrix = calc_dist_matrix_new(sequence, aa_cache)
contact_map_new = np.array((new_matrix < 8.0) & (new_matrix > 0.1))*1

CPU times: user 12.6 ms, sys: 26 µs, total: 12.6 ms
Wall time: 8.54 ms


In [211]:
np.sum(contact_map_old != contact_map_new)

0

In [190]:
new_matrix

array([[0.        , 3.80005654, 5.72770815, 7.52073955, 8.81870349],
       [3.80005654, 0.        , 3.80827122, 6.19648968, 9.02324472],
       [5.72770815, 3.80827122, 0.04419417, 3.83790206, 6.54625967],
       [7.52073955, 6.19648968, 3.83790206, 0.        , 3.85339238],
       [8.81870349, 9.02324472, 6.54625967, 3.85339238, 0.        ]])

In [174]:
old_matrix

array([[0.        , 3.80017614, 5.72765207, 7.52076864, 8.8187027 ],
       [3.80017614, 0.        , 3.80829239, 6.19655752, 9.02326679],
       [5.72765207, 3.80829239, 0.        , 3.8377955 , 6.54619932],
       [7.52076864, 6.19655752, 3.8377955 , 0.        , 3.85341954],
       [8.8187027 , 9.02326679, 6.54619932, 3.85341954, 0.        ]])

In [182]:
x0 = (sequence[0]['CA'].coord)
x1 = (sequence[1]['CA'].coord)

In [183]:
np.linalg.norm(x0 - x1)

3.8001761

In [186]:
np.sqrt(np.sum(np.square(x0)) + np.sum(np.square(x1)) - 2*x0.T@x1)

3.799928